# Демонстрация использования класса `OscillogramActivityFilter`

Этот ноутбук показывает, как использовать класс `OscillogramActivityFilter` из `analysis.activity_filter` для фильтрации "пустых" или неактивных осциллограмм. Фильтр анализирует первую гармонику (H1) выбранных каналов и определяет активность на основе заданных порогов для дельты, стандартного отклонения и максимального значения H1.

Поддерживается два режима анализа для каждого канала:
1.  С использованием нормализованных данных (если предоставлен `OscillogramNormalizer` и коэффициенты, и опция `use_norm_osc` в конфигурации включена).
2.  Анализ "сырых" данных с относительными порогами (если нормализация не используется или не удалась). В этом режиме сигнал сначала проверяется на "чистоту" начального участка.

**Важно**: Этот ноутбук использует образцы файлов из директории `tests/sample_data/`. Для демонстрации эти файлы и конфигурации копируются во временную рабочую директорию (`temp_activity_filter_notebook_data/...`). Убедитесь, что тестовые данные были сгенерированы запуском скрипта `tests/test_data_setup.py`.

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import json 
import sys
import csv # For creating fallback norm_coeffs_for_filter_test.csv

# --- Пути к данным для тестирования ---
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path:
    sys.path.append(module_path)
    
from core.oscillogram import Oscillogram
from normalization.normalization import OscillogramNormalizer
from analysis.activity_filter import OscillogramActivityFilter, ChannelType

base_sample_data_dir = os.path.join(module_path, "tests", "sample_data")
comtrade_samples_for_af_path = os.path.join(base_sample_data_dir, "comtrade_files", "for_activity_filter")
config_samples_path = os.path.join(base_sample_data_dir, "config_files")

# --- Создание временной рабочей директории для этого ноутбука ---
notebook_base_temp_dir_af = "temp_activity_filter_notebook_data"
notebook_source_dir_af = os.path.join(notebook_base_temp_dir_af, "source_oscillograms_af")
notebook_output_dir_af = os.path.join(notebook_base_temp_dir_af, "activity_filter_output")
notebook_config_dir_af = os.path.join(notebook_base_temp_dir_af, "config_files_af")

if os.path.exists(notebook_base_temp_dir_af):
    shutil.rmtree(notebook_base_temp_dir_af)
os.makedirs(notebook_source_dir_af)
os.makedirs(notebook_output_dir_af)
os.makedirs(notebook_config_dir_af)

# --- Копирование необходимых файлов для демонстрации ---
all_files_copied_af = True
# 1. Осциллограммы
af_files_to_copy = [
    "osc_af_active.cfg", "osc_af_active.dat",
    "osc_af_empty_noise.cfg", "osc_af_empty_noise.dat",
    "osc_af_empty_stable.cfg", "osc_af_empty_stable.dat",
    "osc_af_active_fornorm.cfg", "osc_af_active_fornorm.dat"
]
for file_name in af_files_to_copy:
    src_path = os.path.join(comtrade_samples_for_af_path, file_name)
    if os.path.exists(src_path):
        shutil.copy(src_path, notebook_source_dir_af)
    else:
        print(f"ОШИБКА: Исходный файл для теста не найден: {src_path}")
        all_files_copied_af = False
            
# 2. Конфигурационный JSON для ActivityFilter
src_filter_config_json = os.path.join(config_samples_path, "activity_filter_config.json")
dest_filter_config_json_af = os.path.join(notebook_config_dir_af, "activity_filter_config.json")
if os.path.exists(src_filter_config_json):
    shutil.copy(src_filter_config_json, dest_filter_config_json_af)
else:
    print(f"ОШИБКА: activity_filter_config.json не найден в {config_samples_path}.")
    all_files_copied_af = False
    # Создадим дефолтный, если не найден, чтобы ноутбук мог работать
    default_af_config = {
        'channels_to_analyze_patterns': ['I |', 'U |'],
        'current_channel_id_patterns': ['i |'], 'voltage_channel_id_patterns': ['u |'],
        'use_norm_osc': True,
        'norm_yes_phrase': 'YES',
        'thresholds_current_normalized': {'delta': 0.05, 'std_dev': 0.02, 'max_abs': 0.01},
        'thresholds_voltage_normalized': {'delta': 0.03, 'std_dev': 0.015, 'max_abs': 0.02},
        'raw_signal_analysis': {
            'initial_window_check_periods': 1, 'h1_vs_hx_ratio_threshold_U': 5, 
            'h1_vs_hx_ratio_threshold_I': 3,
            'min_initial_h1_amplitude_for_rel_norm': 0.01,
            'thresholds_raw_current_relative': {'delta': 0.2, 'std_dev': 0.1},
            'thresholds_raw_voltage_relative': {'delta': 0.1, 'std_dev': 0.05}
        }, 'verbose': False
    }
    with open(dest_filter_config_json_af, "w", encoding="utf-8") as f: json.dump(default_af_config, f, indent=4)
    print(f"Создан демонстрационный activity_filter_config.json в {notebook_config_dir_af}")

# 3. Файл коэффициентов нормализации (используемый фильтром, если use_norm_osc=True)
src_norm_coeffs_filter_path = os.path.join(config_samples_path, "norm_coeffs_for_filter_test.csv")
dest_norm_coeffs_filter_path_af = os.path.join(notebook_config_dir_af, "norm_coeffs_for_filter_test.csv")
if os.path.exists(src_norm_coeffs_filter_path):
    shutil.copy(src_norm_coeffs_filter_path, dest_norm_coeffs_filter_path_af)
else:
    print(f"ОШИБКА: norm_coeffs_for_filter_test.csv не найден в {config_samples_path}.")
    all_files_copied_af = False
    # Создадим пустой или дефолтный, если не найден
    with open(dest_norm_coeffs_filter_path_af, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(["name", "norm", "1Ub_base"]) # Минимальный заголовок
    print(f"Создан демонстрационный (пустой) norm_coeffs_for_filter_test.csv в {notebook_config_dir_af}")

if not all_files_copied_af:
    print("Не все критичные файлы были скопированы. Демонстрация может быть неполной.")
else:
    print(f"Демонстрационные файлы и конфигурации скопированы во временные директории в: {notebook_base_temp_dir_af}")
    print("Содержимое source_oscillograms_af:")
    for item in os.listdir(notebook_source_dir_af): print(f"- {item}")

## 1. Инициализация `OscillogramActivityFilter`

In [ ]:
normalizer_af = None
# Загружаем конфигурацию из файла
loaded_activity_filter_config = None
if os.path.exists(dest_filter_config_json_af):
    with open(dest_filter_config_json_af, 'r', encoding='utf-8') as f:
        loaded_activity_filter_config = json.load(f)
else:
    print(f"ОШИБКА: Файл конфигурации {dest_filter_config_json_af} не найден. Используется заглушка.")
    # Используем заглушку, если файл не был скопирован или создан
    loaded_activity_filter_config = {'use_norm_osc': False, 'verbose': True} 

if loaded_activity_filter_config.get('use_norm_osc', False) and os.path.exists(dest_norm_coeffs_filter_path_af):
    normalizer_af = OscillogramNormalizer(norm_coef_file_path=dest_norm_coeffs_filter_path_af, is_print_message=True)
else:
    print("Нормализация не будет использоваться (либо use_norm_osc=False, либо файл коэффициентов отсутствует).")
    loaded_activity_filter_config['use_norm_osc'] = False # Убедимся, что фильтр не пытается использовать несуществующий нормализатор

activity_filter = OscillogramActivityFilter(
    config=loaded_activity_filter_config,
    normalizer=normalizer_af,
    show_progress_bars=True # Управляет отображением tqdm прогресс-баров
)

print("OscillogramActivityFilter инициализирован.")
if normalizer_af:
    print("Будет использоваться нормализация (если разрешено для файла в norm_coef.csv).")
else:
    print("Нормализация не будет использоваться.")

## 2. Фильтрация директории (`filter_directory`)
Метод `filter_directory` обрабатывает все осциллограммы в указанной директории и сохраняет список "непустых" (активных) файлов в CSV.

In [ ]:
active_files_csv_path = os.path.join(notebook_output_dir_af, "active_oscillograms.csv")

print(f"\nЗапуск фильтрации директории: {notebook_source_dir_af}")
# activity_filter.verbose = True # Контролируется через show_progress_bars и verbose_logging_method
activity_filter.filter_directory(
    source_dir=notebook_source_dir_af,
    output_csv_path=active_files_csv_path,
    show_progress=True # Параметр для управления tqdm в конкретном методе
)
# activity_filter.verbose = False 

if os.path.exists(active_files_csv_path):
    print(f"\nСодержимое отчета ({active_files_csv_path}):")
    try: 
        from IPython.display import display
        display(pd.read_csv(active_files_csv_path))
    except ImportError:
        print(pd.read_csv(active_files_csv_path))
else:
    print(f"Отчет о непустых файлах не был создан.")

## 3. Проверка активности отдельного файла (`check_activity`)
Можно проверить активность конкретной осциллограммы.

In [ ]:
osc_active_path = os.path.join(notebook_source_dir_af, "osc_af_active.cfg")
osc_empty_path = os.path.join(notebook_source_dir_af, "osc_af_empty_stable.cfg")
osc_active_norm_path = os.path.join(notebook_source_dir_af, "osc_af_active_fornorm.cfg")

# activity_filter.verbose = True # Контролируется через verbose_logging_method в check_activity
try:
    print(f"\nПроверка файла: {os.path.basename(osc_active_path)}")
    osc1 = Oscillogram(osc_active_path)
    is_active1 = activity_filter.check_activity(osc1, verbose_logging_method=True)
    print(f"Результат для {os.path.basename(osc_active_path)}: {'Активен' if is_active1 else 'Пустой'}")

    print(f"\nПроверка файла: {os.path.basename(osc_empty_path)}")
    osc2 = Oscillogram(osc_empty_path)
    is_active2 = activity_filter.check_activity(osc2, verbose_logging_method=True)
    print(f"Результат для {os.path.basename(osc_empty_path)}: {'Активен' if is_active2 else 'Пустой'}")
    
    print(f"\nПроверка файла (требует нормализации): {os.path.basename(osc_active_norm_path)}")
    osc3 = Oscillogram(osc_active_norm_path)
    is_active3 = activity_filter.check_activity(osc3, verbose_logging_method=True)
    print(f"Результат для {os.path.basename(osc_active_norm_path)}: {'Активен' if is_active3 else 'Пустой'}")

except Exception as e:
    print(f"Ошибка при проверке отдельных файлов: {e}")
# activity_filter.verbose = False

In [ ]:
# Очистка временной директории
try:
    if os.path.exists(notebook_base_temp_dir_af):
        shutil.rmtree(notebook_base_temp_dir_af)
        print(f"\nВременная директория {notebook_base_temp_dir_af} удалена.")
    else:
        print(f"\nВременная директория {notebook_base_temp_dir_af} не найдена для удаления.")
except Exception as e:
    print(f"Ошибка при удалении временной директории {notebook_base_temp_dir_af}: {e}")